# Type Coercion in Pydantic
* Notebook completed by Adam Lang
* Date: 4/10/2024
* This notebook will go over utilizing Pydantic's type coercions.


# Type Coercion 101 in Pydantic
* When data is deserialized into a pydantic model, type validation occurs to align the data structures with the pydantic serialization objects.

In [1]:
# import libraries
from pydantic import BaseModel, ValidationError

In [2]:
# instantiate a class
class Coordinates(BaseModel):
  x: float
  y: float

In [4]:
# deserialize data
p1 = Coordinates(x=1.1, y=2.2)
p1

Coordinates(x=1.1, y=2.2)

In [5]:
# model_fields
Coordinates.model_fields

{'x': FieldInfo(annotation=float, required=True),
 'y': FieldInfo(annotation=float, required=True)}

In [6]:
# type of p1.x
print(type(p1.x))

<class 'float'>


Summary:
* So far we have EXACT type matching float = float.
* What if there is NOT an EXACT type match, what do we do?

## Pydantic type coercions
* If a data structure can not be validated, Pydantic will attempt to validate on its own accord.

In [7]:
p2 = Coordinates(x=0, y="1.2")
p2

Coordinates(x=0.0, y=1.2)

In [8]:
# validate coercions
type(p2.x), type(p2.y)

(float, float)

* Thus, we can see that Pydantic validated both as floats.

## Choose level of type coercion in pydantic
* Pydantic docs give us exact conversion tables here: https://docs.pydantic.dev/latest/concepts/conversion_table/
* There are 2 different "modes":
1. Strict -> A strict conversion based on table rules.
2. Lax -> Variable or broad mappings based on input and model rules.


* Example:
    * Coercing `int` -> `str`
          * This only works in "lax mode" but not in strict mode.
          * Refer to the pydantic doc table for specifics.

In [9]:
## create a new model

class Model(BaseModel):
  field: str

In [10]:
# create model instance
Model(field="python")

Model(field='python')

In [11]:
try:
  Model(field=100)
except ValidationError as ex:
  print(ex)

1 validation error for Model
field
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type


Notes:
* ValidationError is helpful here to prevent auto coercions to strings during deserialization.

In [12]:
## new class
class Contact(BaseModel):
  email: str

In [13]:
# utilize a json data struct
initial_json_data = '''
{
  "email": "inewton@principia.com"
}

'''

In [14]:
Contact.model_validate_json(initial_json_data)

Contact(email='inewton@principia.com')

summary: Exact data structure requested was returned.

## Scenario
* What if we are calling an API and they change the data return structure format without informing us? Let's see how pydantic responds to type coercion.

In [15]:
new_json_data = '''
{
  "email": {
    "personal": "inewton@principia.com",
    "work": "issac.newton@themint.com"
  }
}

'''

In [17]:
# try to validate data with try and except
try:
  Contact.model_validate_json(new_json_data)
except ValidationError as ex:
  print(ex)

1 validation error for Contact
email
  Input should be a valid string [type=string_type, input_value={'personal': 'inewton@pri...sac.newton@themint.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type


Summary:
* We can see the `ValidationError` that occurs during JSON deserialization

#### Dictionary coercion scenario

In [18]:
new_data = {
    "email": {
        "personal": "inewton@principia.com",
        "work": "issac.newton@themint.com"
    }
}

In [20]:
Contact(email=str(new_data['email']))

Contact(email="{'personal': 'inewton@principia.com', 'work': 'issac.newton@themint.com'}")

We can see this worked because we put in our own type hinting